In [1]:
liste_med_studie={}


In [2]:
%pip install requests
%pip install beautifulsoup4 time 
%pip install json

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


Kode for å hente informasjon fra alle sidene over studier

In [3]:
#Hente alle studier fra nettsiden
import requests
from bs4 import BeautifulSoup
import time

# Extract studier from all 5 pages: https://fagskolen-viken.no/studier
base_url = "https://fagskolen-viken.no/studier"

try:
    # Loop through all 5 pages
    for page in range(0, 5):
        if page == 0:
            url = base_url
        else:
            url = f"{base_url}?page={page}"
            
        print(f"Scraping page {page}: {url}")
        
        # Send request to get the webpage
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        # Parse HTML content
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find all study articles
        study_articles = soup.find_all('article', class_='study-guide')
        
        print(f"Found {len(study_articles)} studies")
        
        # Extract name and link for each study
        for article in study_articles:
            # Find the link element
            link_element = article.find('a', class_='study-guide__link')
            
            if link_element:
                # Extract the href (relative link)
                relative_link = link_element.get('href')
                # Make it a full URL
                full_link = f"https://fagskolen-viken.no{relative_link}"
                
                # Extract the study name from the h3 span
                title_element = link_element.find('h3')
                if title_element:
                    span_element = title_element.find('span')
                    if span_element:
                        study_name = span_element.get_text().strip()
                        
                        # Add to our dictionary
                        liste_med_studie[study_name] = full_link
                        print(f"Added: {study_name}")
        
        # Small delay between pages
        time.sleep(1)

    print(f"\nTotal studies extracted: {len(liste_med_studie)}")
    
except requests.RequestException as e:
    print(f"Error fetching webpage: {e}")
except Exception as e:
    print(f"Error parsing content: {e}")

Scraping page 0: https://fagskolen-viken.no/studier
Found 15 studies
Added: Administrativ koordinator
Added: Akuttgeriatri
Added: Anlegg
Added: Anvendt maskinlæring
Added: Arbeid med sårbare barn og unge
Added: Automatisering
Added: Avansert prehospital kompetanse
Added: Barn og unge med behov for særskilt tilrettelegging
Added: Barsel og barnepleie
Added: Bedriftsutvikling i frisørfaget
Added: Bilteknikk
Added: BIM-tekniker installasjon
Added: BIM-tekniker konstruksjon
Added: Bygg
Added: Bærekraft i byggenæringen
Scraping page 1: https://fagskolen-viken.no/studier?page=1
Found 15 studies
Added: Cybersikkerhet
Added: Dataanalytiker
Added: Demens og alderspsykiatri
Added: Elkraft
Added: Grunnleggende praktisk lederutdanning
Added: Helhetlig energisystemer
Added: Industriell digitalisert automatisering
Added: Innføring i teknologifag
Added: Innsatsleder Helse
Added: Intensivpleie
Added: ​​IT-systemdrift med cybersikkerhet​
Added: Klima, energi og miljø i bygg (KEM)
Added: Klinisk helsear

Lage en liste over alle studiene i en fil

In [4]:
#Lage listen til en json fil

print(liste_med_studie)

# save in json file
import json
with open('lenker.json', 'w', encoding='utf-8') as f:
    json.dump(liste_med_studie, f, ensure_ascii=False, indent=4)

{'Administrativ koordinator': 'https://fagskolen-viken.no/studier/ledelse/administrativ-koordinator', 'Akuttgeriatri': 'https://fagskolen-viken.no/studier/helse/akuttgeriatri', 'Anlegg': 'https://fagskolen-viken.no/studier/bygg-og-anlegg/anlegg', 'Anvendt maskinlæring': 'https://fagskolen-viken.no/studier/it-og-data/anvendt-maskinlaering', 'Arbeid med sårbare barn og unge': 'https://fagskolen-viken.no/studier/helse/arbeid-med-sarbare-barn-og-unge', 'Automatisering': 'https://fagskolen-viken.no/studier/elektro/automatisering', 'Avansert prehospital kompetanse': 'https://fagskolen-viken.no/studier/helse/avansert-prehospital-kompetanse', 'Barn og unge med behov for særskilt tilrettelegging': 'https://fagskolen-viken.no/studier/barn-og-oppvekst/barn-og-unge-med-behov-saerskilt-tilrettelegging', 'Barsel og barnepleie': 'https://fagskolen-viken.no/studier/helse/barsel-og-barnepleie', 'Bedriftsutvikling i frisørfaget': 'https://fagskolen-viken.no/studier/skjonnhet-og-velvaere/bedriftsutviklin

Kode for å hente informasjon om studier

In [5]:
#Finn studiepoeng
def get_study_points(soup):
    """Extract study points from the BeautifulSoup object"""
    element = soup.find('div', {"class": "field--name-field-study-points"})
    if element:
        return element.get_text().strip()
    return ""


#Finn studiespråk
def get_study_language(soup):
    """Extract study language from the BeautifulSoup object"""
    element = soup.find('div', {"class": "field--name-field-language"})
    if element:
        return element.get_text().strip()
    return ""


#Finn studienivå
def get_study_level(soup):
    """Extract study level from the BeautifulSoup object"""
    element = soup.find('div', {"class": "field--name-field-level"})
    if element:
        return element.get_text().strip()
    return ""


#Finn beskrivelse
def get_study_description(soup):
    """Extract study description from the BeautifulSoup object"""
    element = soup.find('div', {"class": "study-detail--intro__text"})
    if element:
        return element.get_text().strip()
    return ""


#Finn "hvorfor velge dette studiet"
def get_study_why_choose(soup):
    """Extract 'why choose this study' from the BeautifulSoup object"""
    element = soup.find('div', {"class": "study-detail--description__body"})
    if element:
        return element.get_text().strip()
    return ""


#Emnetittel
def get_subject_titles(soup):
    """Extract subject titles from the BeautifulSoup object"""
    titles = []
    seen_titles = set()  # Track already seen titles
    
    # Find the container with the class
    container = soup.find('div', {"class": "study-detail--courses__list"})
    if container:
        # Find all div elements with class study-course__title
        title_elements = container.find_all('div', {"class": "study-course__title"})
        for element in title_elements:
            title_text = element.get_text().strip()
            if title_text and title_text not in seen_titles:  # Only add non-empty and unique titles
                print(title_text)  # Print the actual title text
                titles.append(title_text)
                seen_titles.add(title_text)  # Mark as seen
    return titles


#Finn hva lærer du
def get_what_you_learn(soup):
    """Extract 'what you learn' from the BeautifulSoup object"""
    element = soup.find('div', {"class": "study-detail--courses__body"})
    if element:
        return element.get_text().strip()
    return ""


#Finn undervisning og samlinger
def get_teaching_and_meetings(soup):
    """Extract 'teaching and meetings' from the BeautifulSoup object"""

    is_undervisning = False
    undervisning_og_samlinger = ""
    element = soup.find('div', {"class": "study-detail--courses__body other-info"})
    if not element:  # Check if element exists
        return ""  # Return empty string instead of crashing
    for node in element.contents:
        if node.name == "h3":
            if "Undervisning og samlinger" in node.get_text():
                is_undervisning = True
            else:
                is_undervisning = False
        elif node.name == "p":
            if is_undervisning:
                undervisning_og_samlinger += node.get_text().strip() + "\n"
    return undervisning_og_samlinger.strip()
#print(obligatorisk_oppmøte.strip())


#Finn obligatorisk oppmøte  
def get_mandatory_attendance(soup):

    is_oppmøte = False
    obligatorisk_oppmøte = ""
    element = soup.find('div', {"class": "study-detail--courses__body other-info"})
    if not element:  # Check if element exists
        return ""  # Return empty string instead of crashing
    for node in element.contents:
        if node.name == "h3":
            if "Obligatorisk oppmøte" in node.get_text():
                is_oppmøte = True
            else:
                is_oppmøte = False
        elif node.name == "p":
            if is_oppmøte:
                obligatorisk_oppmøte += node.get_text().strip() + "\n"
    return(obligatorisk_oppmøte.strip())


#Politiattest
def get_police_attest(soup):

    is_politi = False
    politi_attest = ""
    element = soup.find('div', {"class": "study-detail--courses__body other-info"})
    if not element:  # Check if element exists
        return ""  # Return empty string instead of crashing
    for node in element.contents:
        if node.name == "h3":
            if "Politiattest" in node.get_text():
                is_politi = True
            else:
                is_politi = False
        elif node.name == "p":
            if is_politi:
                politi_attest += node.get_text().strip() + "\n"
    return(politi_attest.strip())


#Karrieremuligheter
def get_carrer_possibilities(soup):
    """Extract all subject titles from the BeautifulSoup object"""
    elements = soup.find_all('div', {"class": "study-detail--skill-and-jobs"})
    for element in elements:
        return element.get_text().strip()
    return ""

In [6]:
# Her går vi til serveren, og spør etter HTML dokumentet.
# Dette er det som blir vist frem i nettleseren.
# Det er tilgjengelig i response.content hvis kallet gikk bra. Egentlig så burde vi sjekke at det gikk bra før vi bruker response.content.

studie = "Administrativ koordinator"
print(liste_med_studie[studie])    # Print lenken
response = requests.get(liste_med_studie[studie])
soup = BeautifulSoup(response.content, 'html.parser')


https://fagskolen-viken.no/studier/ledelse/administrativ-koordinator


Teste kode for å hente informasjon fra et studie uten å gjøre nettverkskall. Bruker studiet i cellen over

In [7]:
#Test koden uten å gjøre nettverkskall
#print(get_study_points(soup))
#print(get_study_language(soup))
#print(get_study_level(soup))
#print(get_study_description(soup))
#print(get_study_why_choose(soup))
#print(get_subject_titles(soup))
#print(get_what_you_learn(soup))
#print(get_teaching_and_meetings(soup))
#print(get_mandatory_attendance(soup))
#print(get_police_attest(soup))
#print(get_carrer_possibilities(soup))

Lage kode for å autofylle ut studieinfo

In [8]:
# Load the study links from JSON file
import json

# Load the study links
with open('lenker.json', 'r', encoding='utf-8') as f:
    study_links = json.load(f)

print(f"Loaded {len(study_links)} studies from lenker.json")

# Template for study data
template = {
    "title": "",
    "link": "",
    "beskrivelse": "",
    "studiepoeng": "",
    "undervisningsspråk": "",
    "nivå": "",
    "hvorfor velge studiet": "",
    "hva lærer du": "",
    "emne": "",
    "undervisning og samlinger": "",
    "obligatorisk oppmøte": "",
    "politiattest": "",
    "karrieremuligheter": ""
}

Loaded 62 studies from lenker.json


Kode for å hente informasjon fra et enkelt studie og lage system på hvordan informasjonen blir vist

In [9]:
# Function to fetch all study data for a single study
def fetch_study_data(study_name, study_url):
    """Fetch all data for a single study using the scraping functions"""
    import time
    
    print(f"Fetching data for: {study_name}")
    
    try:
        # Get the webpage
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(study_url, headers=headers)
        response.raise_for_status()
        
        # Parse HTML
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Create study data using template
        study_data = template.copy()
        
        # Fill in the data using our functions
        study_data["title"] = study_name
        study_data["link"] = study_url
        study_data["beskrivelse"] = get_study_description(soup)
        study_data["studiepoeng"] = get_study_points(soup) 
        study_data["undervisningsspråk"] = get_study_language(soup)
        study_data["nivå"] = get_study_level(soup)
        study_data["hvorfor velge studiet"] = get_study_why_choose(soup)
        study_data["hva lærer du"] = get_what_you_learn(soup)
        study_data["emne"] = get_subject_titles(soup) 
        study_data["undervisning og samlinger"] = get_teaching_and_meetings(soup)
        study_data["obligatorisk oppmøte"] = get_mandatory_attendance(soup)
        study_data["politiattest"] = get_police_attest(soup)
        study_data["karrieremuligheter"] = get_carrer_possibilities(soup)
        
        print(f"✓ Successfully fetched data for {study_name}")
        time.sleep(1)  # Be nice to the server
        
        return study_data
        
    except Exception as e:
        print(f"❌ Error fetching data for {study_name}: {e}")
        return None

Kode for å sjekke at den kan hente informasjon fra et studie

In [10]:
# BASELINE: Test with one study first
baseline_study = "Akuttgeriatri"
baseline_url = study_links[baseline_study]

print(f"Creating baseline with: {baseline_study}")
baseline_data = fetch_study_data(baseline_study, baseline_url)

if baseline_data:
    print("\n=== BASELINE RESULT ===")
    import pprint
    pprint.pprint(baseline_data, width=100, depth=2)

Creating baseline with: Akuttgeriatri
Fetching data for: Akuttgeriatri
Anatomi, fysiologi og mikrobiologi
Akuttmedisinske sykdommer og tilstander
Handlingskompetanse akuttgeriatri
✓ Successfully fetched data for Akuttgeriatri

=== BASELINE RESULT ===
{'beskrivelse': 'Akuttgeriatri er videreutdanningen for deg som jobber med eldre pasienter og '
                'ønsker faglig trygghet når helsetilstanden endrer seg raskt. Du får verktøyene '
                'til å oppdage forverring tidlig, vurdere situasjonen strukturert og bidra til '
                'trygg, verdig og pasientsikker oppfølging – i både kommune- og '
                'spesialisthelsetjenesten.',
 'emne': ['Anatomi, fysiologi og mikrobiologi',
          'Akuttmedisinske sykdommer og tilstander',
          'Handlingskompetanse akuttgeriatri'],
 'hva lærer du': 'Du bygger spisskompetanse i akuttgeriatri, slik at du kan oppdage tidlige tegn '
                 'på forverring hos eldre og bidra til rask, trygg og faglig forsva

Kode for å hente informasjon fra alle studiene

In [12]:
# FULL AUTOMATION: Process all 62 studies
print(f"Processing all {len(study_links)} studies...")

all_studies_data = []
failed_studies = []

for i, (study_name, study_url) in enumerate(study_links.items(), 1):
    print(f"\n[{i}/{len(study_links)}] Processing: {study_name}")
    
    study_data = fetch_study_data(study_name, study_url)
    
    if study_data:
        all_studies_data.append(study_data)
    else:
        failed_studies.append(study_name)
    
    # Progress update every 10 studies
    if i % 10 == 0:
        print(f"\n=== PROGRESS UPDATE ===")
        print(f"Completed: {len(all_studies_data)}")
        print(f"Failed: {len(failed_studies)}")
        print(f"Remaining: {len(study_links) - i}")

print(f"\n=== FINAL RESULTS ===")
print(f"Successfully processed: {len(all_studies_data)} studies")
print(f"Failed to process: {len(failed_studies)} studies")

if failed_studies:
    print(f"Failed studies: {', '.join(failed_studies)}")

# Save to JSON file
with open('studie.json', 'w', encoding='utf-8') as f:
    json.dump(all_studies_data, f, ensure_ascii=False, indent=2)
    
print(f"\n✓ All study data saved to 'studie.json'")
print(f"Total studies in file: {len(all_studies_data)}")

Processing all 62 studies...

[1/62] Processing: Administrativ koordinator
Fetching data for: Administrativ koordinator
Organisasjonsforståelse
Kommunikasjon, inkludert digitale verktøy
Administrativ lederstøtte, inkludert økonomi og personalarbeid
Endring og utvikling
Kontordrift/ praktisk lederstøtte
Hovedprosjekt (faglig fordypning)
✓ Successfully fetched data for Administrativ koordinator

[2/62] Processing: Akuttgeriatri
Fetching data for: Akuttgeriatri
Anatomi, fysiologi og mikrobiologi
Akuttmedisinske sykdommer og tilstander
Handlingskompetanse akuttgeriatri
✓ Successfully fetched data for Akuttgeriatri

[3/62] Processing: Anlegg
Fetching data for: Anlegg
Realfaglige redskap
Kommunikasjon
Ledelse, økonomi og markedsføring (LØM)
Grunnleggende bygg og anlegg
Digital byggeprosess
Anleggsproduksjon
Geomatikk og BIM anlegg
Konstruksjonslære anlegg
Anleggsteknikk
Hovedprosjekt
✓ Successfully fetched data for Anlegg

[4/62] Processing: Anvendt maskinlæring
Fetching data for: Anvendt ma

Kode for å sjekke at den kan hente informasjon om emner fra et studie

In [13]:
#Utgangspunkt ekstrakt av emnelinker fra studienettsiden

from urllib.parse import urljoin

from urllib.request import Request, urlopen
from urllib.parse import urljoin

def extract_emnelinker(url) -> dict[str, str]:
    """ Extract emnelinker from the page
    returns {emne_name: emne_link}
    """

    req = Request(url, headers={"User-Agent": "Mozilla/5.0"})
    html = urlopen(req).read().decode("utf-8", errors="ignore")
    soup = BeautifulSoup(html, "lxml")

    emner = []
    
    # Find all course links in the "study-detail--courses" section
    for a in soup.select('.study-detail--courses a.study-course__link'):
        href = a.get("href", "").strip()
        if not href:
            continue
        
        # Get the course title
        title_el = a.select_one(".study-course__title")
        if title_el:
            title = title_el.get_text(strip=True)
            emner.append({"emne_name": title, "href": href})
    
    return emner

print(extract_emnelinker("https://fagskolen-viken.no/studier/helse/arbeid-med-sarbare-barn-og-unge"))

[{'emne_name': 'Helsetjenester og rettigheter til sårbare barn og unge', 'href': 'https://studiekatalog.edutorium.no/viken/nb/emne/00001EIHOABF-25/2025-host'}, {'emne_name': 'Familiesentrert omsorg, kommunikasjon og samhandling', 'href': 'https://studiekatalog.edutorium.no/viken/nb/emne/00002EIHOABF--25/2025-host'}, {'emne_name': 'Psykisk helse hos barn og unge', 'href': 'https://studiekatalog.edutorium.no/viken/nb/emne/00003EIHOABF-25/2026-var'}, {'emne_name': 'Barn og unge med nedsatt funksjonsevne', 'href': 'https://studiekatalog.edutorium.no/viken/nb/emne/00004EIHOABF-25/2026-host'}, {'emne_name': 'Psykisk utviklingshemming hos barn og unge', 'href': 'https://studiekatalog.edutorium.no/viken/nb/emne/00005EIHOABF--25/2026-host'}, {'emne_name': 'Aktivitet og deltakelse, velferdsteknologi og hjelpemidler', 'href': 'https://studiekatalog.edutorium.no/viken/nb/emne/00006EIHOABF-25/2027-var'}, {'emne_name': 'Helsetjenester og rettigheter til sårbare barn og unge', 'href': 'https://studie

Kode for å sjekke om den kan hente emner fra alle studiene være i json filen

In [14]:
# Extract all subjects from all study URLs and save to JSON (including duplicates)

# Load study links from JSON file
with open('lenker.json', 'r', encoding='utf-8') as f:
    study_links = json.load(f)

all_subjects = []
failed_urls = []

print(f"Processing {len(study_links)} study URLs to extract all subjects...")

for i, (study_name, study_url) in enumerate(study_links.items(), 1):
    print(f"[{i}/{len(study_links)}] Extracting subjects from: {study_name}")
    
    try:
        # Extract subjects from this study page
        emner = extract_emnelinker(study_url)
        
        # Add each subject to our list with study context
        for emne in emner:
            subject_entry = {
                "emne_name": emne["emne_name"],
                "emne_link": urljoin(study_url, emne["href"]),
                "from_study": study_name,
                "study_url": study_url
            }
            all_subjects.append(subject_entry)
        
        print(f"  ✓ Found {len(emner)} subjects")
        
    except Exception as e:
        print(f"  ❌ Error processing {study_name}: {e}")
        failed_urls.append(study_name)
    
    # Small delay to be nice to the server
    time.sleep(0.5)

print(f"\n=== EXTRACTION COMPLETE ===")
print(f"Total subjects found: {len(all_subjects)}")
print(f"Failed URLs: {len(failed_urls)}")

if failed_urls:
    print(f"Failed studies: {', '.join(failed_urls)}")

# Save all subjects to JSON file (including duplicates)
with open('alle_emner.json', 'w', encoding='utf-8') as f:
    json.dump(all_subjects, f, ensure_ascii=False, indent=2)

print(f"\n✓ All subjects saved to 'alle_emner.json'")
print(f"File contains {len(all_subjects)} subject entries (including duplicates)")

# Show sample of what was extracted
if all_subjects:
    print(f"\nSample entries:")
    for i in range(min(3, len(all_subjects))):
        print(f"  {i+1}. {all_subjects[i]['emne_name']} (from {all_subjects[i]['from_study']})")

Processing 62 study URLs to extract all subjects...
[1/62] Extracting subjects from: Administrativ koordinator
  ✓ Found 6 subjects
[2/62] Extracting subjects from: Akuttgeriatri
  ✓ Found 0 subjects
[3/62] Extracting subjects from: Anlegg
  ✓ Found 20 subjects
[4/62] Extracting subjects from: Anvendt maskinlæring
  ✓ Found 6 subjects
[5/62] Extracting subjects from: Arbeid med sårbare barn og unge
  ✓ Found 17 subjects
[6/62] Extracting subjects from: Automatisering
  ✓ Found 19 subjects
[7/62] Extracting subjects from: Avansert prehospital kompetanse
  ✓ Found 9 subjects
[8/62] Extracting subjects from: Barn og unge med behov for særskilt tilrettelegging
  ✓ Found 15 subjects
[9/62] Extracting subjects from: Barsel og barnepleie
  ✓ Found 6 subjects
[10/62] Extracting subjects from: Bedriftsutvikling i frisørfaget
  ✓ Found 3 subjects
[11/62] Extracting subjects from: Bilteknikk
  ✓ Found 14 subjects
[12/62] Extracting subjects from: BIM-tekniker installasjon
  ✓ Found 3 subjects
[13

Kode for å hente informasjon om alle emnene til alle studiene 

In [16]:
# Function to fetch HTML content from subject URLs
def fetch_subject_html(emne_url):
    """Fetch and parse HTML content from a subject URL"""
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(emne_url, headers=headers)
        response.raise_for_status()
        
        # Parse HTML content
        soup = BeautifulSoup(response.content, 'html.parser')
        return soup
        
    except Exception as e:
        print(f"Error fetching {emne_url}: {e}")
        return None

# Load subject links from JSON file
with open('alle_emner.json', 'r', encoding='utf-8') as f:
    all_subjects = json.load(f)

print(f"Loaded {len(all_subjects)} subject entries")

# Test with one subject URL
test_subject = all_subjects[0]
print(f"\nTesting with: {test_subject['emne_name']}")
print(f"URL: {test_subject['emne_link']}")

# Fetch HTML for the test subject
soup = fetch_subject_html(test_subject['emne_link'])

if soup:
    print("✓ Successfully fetched and parsed HTML")
    print("Ready to extract data using the parsing functions")
else:
    print("❌ Failed to fetch HTML")

Loaded 579 subject entries

Testing with: Organisasjonsforståelse
URL: https://studiekatalog.edutorium.no/viken/nb/emne/02AL66A-EILADK/2025-host
✓ Successfully fetched and parsed HTML
Ready to extract data using the parsing functions


Kode for å hente ønsket informasjon om et emen

In [17]:
#Lag definisjoner for alle variabler vi ønsker om emnet

#Emnekode
def get_subject_code(soup):
    """Extract subject code from the BeautifulSoup object"""
    # Find the facts-label that contains "Emnekode"
    labels = soup.find_all('div', {"class": "facts-label"})
    for label in labels:
        if "Emnekode" in label.get_text():
            # Find the next sibling facts-item div
            facts_item = label.find_next_sibling('div', {"class": "facts-item"})
            if facts_item:
                return facts_item.get_text().strip()
    return ""

#Studieprogram
def get_subject_program(soup):
    """Extract subject program from the BeautifulSoup object"""
    # Find the facts-label that contains "Studieprogram"
    labels = soup.find_all('div', {"class": "facts-label"})
    for label in labels:
        if "Studieprogram" in label.get_text():
            # Find the next sibling facts-item div
            facts_item = label.find_next_sibling('div', {"class": "facts-item"})
            if facts_item:
                return facts_item.get_text().strip()
    return ""

#Emnepoeng
def get_subject_score(soup):
    """Extract subject score from the BeautifulSoup object"""
    # Find the facts-label that contains "Studiepoeng"
    labels = soup.find_all('div', {"class": "facts-label"})
    for label in labels:
        if "Studiepoeng" in label.get_text():
            # Find the next sibling facts-item div
            facts_item = label.find_next_sibling('div', {"class": "facts-item"})
            if facts_item:
                return facts_item.get_text().strip()
    return ""

#Studiested
def get_subject_location(soup):
    """Extract subject location from the BeautifulSoup object"""
    # Find the facts-label that contains "Studiested"
    labels = soup.find_all('div', {"class": "facts-label"})
    for label in labels:
        if "Studiested" in label.get_text():
            # Find the next sibling facts-item div
            facts_item = label.find_next_sibling('div', {"class": "facts-item"})
            if facts_item:
                return facts_item.get_text().strip()
    
    return ""

#Emnenivå
def get_subject_skill(soup):
    """Extract subject score from the BeautifulSoup object"""
    # Find the facts-label that contains "Studienivå"
    labels = soup.find_all('div', {"class": "facts-label"})
    for label in labels:
        if "Studienivå" in label.get_text():
            # Find the next sibling facts-item div
            facts_item = label.find_next_sibling('div', {"class": "facts-item"})
            if facts_item:
                return facts_item.get_text().strip()
    return ""

#Emneansvarlig
def get_subject_responsible(soup):
    """Extract subject responsible from the BeautifulSoup object"""
    # Find the facts-label that contains "Emneansvarlig"
    labels = soup.find_all('div', {"class": "facts-label"})
    for label in labels:
        if "Emneansvarlig" in label.get_text():
            # Find the next sibling facts-item div
            facts_item = label.find_next_sibling('div', {"class": "facts-item"})
            if facts_item:
                return facts_item.get_text().strip()
    return ""

#Læringsutbytte
#-Kunnskap
def get_subject_learning_objectives(soup):
    """Extract all subject titles from the BeautifulSoup object"""
    elements = soup.find_all('div', {"class": "field-learning-outcome-knowledge"})
    for element in elements:
        return element.get_text().strip()
    return ""
#-Ferdigheter
def get_subject_learning_skills(soup):
    """Extract all subject titles from the BeautifulSoup object"""
    elements = soup.find_all('div', {"class": "field-learning-outcome-skills"})
    for element in elements:
        return element.get_text().strip()
    return ""
#-Generell kompetanse
def get_subject_learning_competence(soup):
    """Extract all subject titles from the BeautifulSoup object"""
    elements = soup.find_all('div', {"class": "field-learning-outcome-reflec"})
    for element in elements:
        return element.get_text().strip()
    return ""

#Test at den får riktige verdier for hvert enkelt punkt

#print(get_subject_code(soup))
#print(get_subject_program(soup))
#print(get_subject_score(soup))
#print(get_subject_skill(soup))
#print(get_subject_location(soup))
#print(get_subject_responsible(soup))
#print(get_subject_learning_objectives(soup))
#print(get_subject_learning_skills(soup))
#print(get_subject_learning_competence(soup))

Kode for å hente all ønsket informasjon om emner fra alle studier og sette det i ønsket system

In [18]:
# Create a function to fetch all subject data for a single subject
def fetch_subject_data(subject_name, subject_url, from_study):
    """Fetch all data for a single subject using the subject scraping functions"""
    
    print(f"Fetching subject data for: {subject_name} (from {from_study})")
    
    try:
        # Get the webpage
        soup = fetch_subject_html(subject_url)
        
        if not soup:
            return None
        
        # Create subject data dictionary
        subject_data = {
            "emne_name": subject_name,
            "emne_link": subject_url,
            "from_study": from_study,
            "emnekode": get_subject_code(soup),
            "studieprogram": get_subject_program(soup),
            "studiepoeng": get_subject_score(soup),
            "studiested": get_subject_location(soup),
            "studienivå": get_subject_skill(soup),
            "emneansvarlig": get_subject_responsible(soup),
            "læringsutbytte_kunnskap": get_subject_learning_objectives(soup),
            "læringsutbytte_ferdigheter": get_subject_learning_skills(soup),
            "læringsutbytte_generell_kompetanse": get_subject_learning_competence(soup)
        }
        
        print(f"✓ Successfully fetched subject data for {subject_name}")
        time.sleep(0.5)  # Be nice to the server
        
        return subject_data
        
    except Exception as e:
        print(f"❌ Error fetching subject data for {subject_name}: {e}")
        return None

# Process all subjects from alle_emner.json
print(f"Processing all {len(all_subjects)} subjects to extract detailed data...")

enriched_subjects = []
failed_subjects = []

for i, subject in enumerate(all_subjects, 1):
    print(f"\n[{i}/{len(all_subjects)}] Processing: {subject['emne_name']}")
    
    # Skip invalid URLs (like nginx URLs that are not accessible)
    if 'nginx' in subject['emne_link'] or 'edit' in subject['emne_link']:
        print(f"  ⚠️ Skipping invalid URL: {subject['emne_link']}")
        # Add original subject data without enrichment
        enriched_subjects.append(subject)
        continue
    
    # Fetch detailed subject data
    detailed_subject = fetch_subject_data(
        subject['emne_name'], 
        subject['emne_link'], 
        subject['from_study']
    )
    
    if detailed_subject:
        # Also include original study_url
        detailed_subject['study_url'] = subject['study_url']
        enriched_subjects.append(detailed_subject)
    else:
        failed_subjects.append(subject['emne_name'])
        # Add original subject data as fallback
        enriched_subjects.append(subject)
    
    # Progress update every 50 subjects
    if i % 50 == 0:
        print(f"\n=== PROGRESS UPDATE ===")
        print(f"Completed: {len(enriched_subjects)}")
        print(f"Failed: {len(failed_subjects)}")
        print(f"Remaining: {len(all_subjects) - i}")

print(f"\n=== FINAL RESULTS ===")
print(f"Successfully processed: {len(enriched_subjects) - len(failed_subjects)} subjects")
print(f"Failed to process: {len(failed_subjects)} subjects")

if failed_subjects:
    print(f"Failed subjects: {', '.join(failed_subjects[:10])}...")  # Show first 10 only

# Save enriched data back to JSON file
with open('alle_emner_enriched.json', 'w', encoding='utf-8') as f:
    json.dump(enriched_subjects, f, ensure_ascii=False, indent=2)
    
print(f"\n✓ Enriched subject data saved to 'alle_emner_enriched.json'")
print(f"Total subjects in file: {len(enriched_subjects)}")

# Show sample of enriched data
if enriched_subjects:
    print(f"\nSample enriched subject:")
    sample = enriched_subjects[0]
    for key, value in sample.items():
        if isinstance(value, str) and len(value) > 100:
            print(f"  {key}: {value[:100]}...")
        else:
            print(f"  {key}: {value}")

Processing all 579 subjects to extract detailed data...

[1/579] Processing: Organisasjonsforståelse
Fetching subject data for: Organisasjonsforståelse (from Administrativ koordinator)
✓ Successfully fetched subject data for Organisasjonsforståelse

[2/579] Processing: Kommunikasjon, inkludert digitale verktøy
Fetching subject data for: Kommunikasjon, inkludert digitale verktøy (from Administrativ koordinator)
✓ Successfully fetched subject data for Kommunikasjon, inkludert digitale verktøy

[3/579] Processing: Administrativ lederstøtte, inkludert økonomi og personalarbeid
Fetching subject data for: Administrativ lederstøtte, inkludert økonomi og personalarbeid (from Administrativ koordinator)
✓ Successfully fetched subject data for Administrativ lederstøtte, inkludert økonomi og personalarbeid

[4/579] Processing: Endring og utvikling
Fetching subject data for: Endring og utvikling (from Administrativ koordinator)
✓ Successfully fetched subject data for Endring og utvikling

[5/579] P

Kode for å sjekke om noen av URL'ene er feil fra json filen vår over alle emner

In [27]:
# Søk etter "nginx" i emnelinkene fra dokumentet "alle_emner_enriched.json"

# Load the enriched subjects data
with open('alle_emner_enriched.json', 'r', encoding='utf-8') as f:
    enriched_subjects = json.load(f)

print(f"Loaded {len(enriched_subjects)} subjects from alle_emner_enriched.json")

# Search for entries with "nginx" in any URL fields
nginx_subjects = []

for subject in enriched_subjects:
    # Check both emne_link and study_url for "nginx"
    emne_link = subject.get('emne_link', '').lower()
    study_url = subject.get('study_url', '').lower()
    
    if 'nginx' in emne_link or 'nginx' in study_url:
        nginx_entry = {
            'subject_name': subject.get('emne_name', ''),
            'subject_url': subject.get('emne_link', ''),
            'from_study': subject.get('from_study', ''),
            'study_url': subject.get('study_url', '')
        }
        nginx_subjects.append(nginx_entry)

print(f"\n=== NGINX URLS FOUND ===")
print(f"Total subjects with nginx URLs: {len(nginx_subjects)}")

if nginx_subjects:
    print(f"\nList of subjects with nginx URLs:")
    for i, subject in enumerate(nginx_subjects, 1):
        print(f"\n{i}. Subject: {subject['subject_name']}")
        print(f"   Subject URL: {subject['subject_url']}")
        print(f"   From Study: {subject['from_study']}")
        print(f"   Study URL: {subject['study_url']}")
        
        # Identify which URL contains nginx
        if 'nginx' in subject['subject_url'].lower():
            print("   ⚠️ nginx found in subject URL")
        if 'nginx' in subject['study_url'].lower():
            print("   ⚠️ nginx found in study URL")
else:
    print("✅ No nginx URLs found in the data")

Loaded 579 subjects from alle_emner_enriched.json

=== NGINX URLS FOUND ===
Total subjects with nginx URLs: 2

List of subjects with nginx URLs:

1. Subject: Psykisk helse hos barn og unge
   Subject URL: https://nginx-fagskolen-viken-cloud-feature-new-study-template.frontkom1.amazee.io/node/1602/edit
   From Study: Arbeid med sårbare barn og unge
   Study URL: https://fagskolen-viken.no/studier/helse/arbeid-med-sarbare-barn-og-unge
   ⚠️ nginx found in subject URL

2. Subject: Velferdsteknologi
   Subject URL: https://nginx-fagskolen-viken-cloud-feature-new-study-template.frontkom1.amazee.io/node/1274/edit
   From Study: Observasjons- og vurderingskompetanse i helsetjenesten
   Study URL: https://fagskolen-viken.no/studier/helse/observasjons-og-vurderingskompetanse-i-helsetjenesten
   ⚠️ nginx found in subject URL


Kode for å manuelt hente data fra emner som den ikke tidligere klarte å hente informasjon fra grunnet feilaktig link for deretter å lage en ny fil med disse

In [21]:
# Kode for å manuelt hente data fra emner som mangler eller har feil i "alle_emner_enriched.json"

# CONFIGURE THESE VALUES
subject_name = "Praktisk ombruk og sirkulærøkonomi"  # Enter the subject name here
subject_url = "https://fagskolen-viken.no/studier/bygg-og-anlegg/baerekraft-i-byggenaeringen/praktisk-ombruk-og-sirkulaerokonomi"  # Enter the subject URL here
from_study = "Bærekraft i byggenæringen"  # Enter which study this subject is from
study_url = "https://fagskolen-viken.no/studier/bygg-og-anlegg/baerekraft-i-byggenaeringen"  # Enter the main study URL

print(f"Fetching data for specific subject:")
print(f"Subject: {subject_name}")
print(f"URL: {subject_url}")
print(f"From study: {from_study}")
print("-" * 50)

# Fetch detailed subject data
detailed_subject = fetch_subject_data(subject_name, subject_url, from_study)

if detailed_subject:
    # Add the study_url to match enriched format exactly
    detailed_subject['study_url'] = study_url
    
    print(f"\n✅ SUCCESS: Fetched data for {subject_name}")
    
    # Display the fetched data in enriched format
    print(f"\n📋 SUBJECT DATA (enriched format):")
    print("{")
    for key, value in detailed_subject.items():
        if isinstance(value, str) and len(value) > 100:
            print(f'  "{key}": "{value[:100]}...",')
        else:
            print(f'  "{key}": "{value}",')
    print("}")
    
    # Load existing data from alle_emner_enriched_mangler.json or create new list
    mangler_filename = "alle_emner_enriched_mangler.json"
    
    try:
        with open(mangler_filename, 'r', encoding='utf-8') as f:
            existing_subjects = json.load(f)
        print(f"\n📂 Loaded {len(existing_subjects)} existing subjects from {mangler_filename}")
    except FileNotFoundError:
        existing_subjects = []
        print(f"\n📂 Creating new file {mangler_filename}")
    
    # Check if this subject already exists (by name and URL)
    subject_exists = False
    for i, existing in enumerate(existing_subjects):
        if (existing.get('emne_name') == subject_name and 
            existing.get('emne_link') == subject_url):
            # Update existing entry
            existing_subjects[i] = detailed_subject
            subject_exists = True
            print(f"🔄 Updated existing subject: {subject_name}")
            break
    
    if not subject_exists:
        # Add new subject to list
        existing_subjects.append(detailed_subject)
        print(f"➕ Added new subject: {subject_name}")
    
    # Save updated list back to file
    with open(mangler_filename, 'w', encoding='utf-8') as f:
        json.dump(existing_subjects, f, ensure_ascii=False, indent=2)
    
    print(f"\n💾 Updated {mangler_filename}")
    print(f"📊 Total subjects in file: {len(existing_subjects)}")
    
    # Show current contents summary
    print(f"\n📋 Current subjects in {mangler_filename}:")
    for i, subj in enumerate(existing_subjects, 1):
        print(f"  {i}. {subj['emne_name']} (from {subj['from_study']})")
    
else:
    print(f"❌ FAILED: Could not fetch data for {subject_name}")
    print("Check the URL and try again")

Fetching data for specific subject:
Subject: Praktisk ombruk og sirkulærøkonomi
URL: https://fagskolen-viken.no/studier/bygg-og-anlegg/baerekraft-i-byggenaeringen/praktisk-ombruk-og-sirkulaerokonomi
From study: Bærekraft i byggenæringen
--------------------------------------------------
Fetching subject data for: Praktisk ombruk og sirkulærøkonomi (from Bærekraft i byggenæringen)
✓ Successfully fetched subject data for Praktisk ombruk og sirkulærøkonomi

✅ SUCCESS: Fetched data for Praktisk ombruk og sirkulærøkonomi

📋 SUBJECT DATA (enriched format):
{
  "emne_name": "Praktisk ombruk og sirkulærøkonomi",
  "emne_link": "https://fagskolen-viken.no/studier/bygg-og-anlegg/baerekraft-i-byggenaeringen/praktisk-ombruk-og-sir...",
  "from_study": "Bærekraft i byggenæringen",
  "emnekode": "",
  "studieprogram": "",
  "studiepoeng": "",
  "studiested": "",
  "studienivå": "",
  "emneansvarlig": "",
  "læringsutbytte_kunnskap": "",
  "læringsutbytte_ferdigheter": "",
  "læringsutbytte_generell_